# 🎯 Generate Predictions - Advanced Ensemble
## Creazione submission per Kaggle utilizzando l'ensemble ottimizzato

### Processo:
1. Caricare i modelli addestrati
2. Processare il prediction_mapping per creare le feature necessarie
3. Generare predizioni con ensemble
4. Creare file submission nel formato corretto

In [ ]:
import pandas as pd
import numpy as np
import pickle
import warnings
warnings.filterwarnings('ignore')

# ML Libraries
import lightgbm as lgb
from catboost import CatBoostRegressor
import xgboost as xgb

# Deep Learning
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler

print("📦 Librerie caricate con successo!")

## 1. Caricamento Modelli e Artefatti

In [ ]:
print("📂 Caricamento modelli...\n")

# Carica feature columns
with open('../models/feature_columns.pkl', 'rb') as f:
    feature_cols = pickle.load(f)
print(f"✅ Feature columns: {len(feature_cols)}")

# Carica scaler
with open('../models/scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)
print(f"✅ Scaler caricato")

# Carica ensemble weights
with open('../models/ensemble_weights.pkl', 'rb') as f:
    ensemble_weights = pickle.load(f)
print(f"✅ Ensemble weights caricati:")
for name, weight in ensemble_weights.items():
    print(f"   {name}: {weight:.4f}")

# Carica modelli
print(f"\n🔧 Caricamento modelli...")

# LightGBM
lgb_model = lgb.Booster(model_file='../models/lightgbm_quantile_model.txt')
print(f"✅ LightGBM caricato")

# CatBoost
catboost_model = CatBoostRegressor()
catboost_model.load_model('../models/catboost_quantile_model.cbm')
print(f"✅ CatBoost caricato")

# XGBoost
xgb_model = xgb.XGBRegressor()
xgb_model.load_model('../models/xgboost_quantile_model.json')
print(f"✅ XGBoost caricato")

# Neural Network
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class QuantileNN(nn.Module):
    def __init__(self, input_dim, hidden_dims=[512, 256, 128, 64]):
        super(QuantileNN, self).__init__()
        
        layers = []
        prev_dim = input_dim
        
        for hidden_dim in hidden_dims:
            layers.append(nn.Linear(prev_dim, hidden_dim))
            layers.append(nn.BatchNorm1d(hidden_dim))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(0.3))
            prev_dim = hidden_dim
        
        layers.append(nn.Linear(prev_dim, 1))
        
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.network(x).squeeze()

nn_model = QuantileNN(input_dim=len(feature_cols)).to(device)
nn_model.load_state_dict(torch.load('../models/best_nn_model.pth', map_location=device))
nn_model.eval()
print(f"✅ Neural Network caricato (device: {device})")

print(f"\n🎉 Tutti i modelli caricati con successo!")

## 2. Caricamento e Preparazione Dati

In [ ]:
print("📊 Caricamento dati storici e prediction mapping...\n")

# Carica dati processati (per estrarre statistiche storiche)
historical_data = pd.read_csv('../data/advanced_features.csv')
historical_data['date_arrival'] = pd.to_datetime(historical_data['date_arrival'])

print(f"✅ Dati storici: {historical_data.shape}")
print(f"   Range: {historical_data['date_arrival'].min()} → {historical_data['date_arrival'].max()}")

# Carica prediction mapping
prediction_mapping = pd.read_csv('../../data/prediction_mapping.csv')
prediction_mapping['forecast_start_date'] = pd.to_datetime(prediction_mapping['forecast_start_date'])
prediction_mapping['forecast_end_date'] = pd.to_datetime(prediction_mapping['forecast_end_date'])

print(f"\n✅ Prediction mapping: {prediction_mapping.shape}")
print(f"   IDs unici: {prediction_mapping['ID'].nunique()}")
print(f"   rm_ids unici: {prediction_mapping['rm_id'].nunique()}")

print("\n📋 Prime righe prediction mapping:")
display(prediction_mapping.head(10))

## 3. Feature Engineering per Prediction Set

In [ ]:
print("🔧 Creazione feature per prediction set...\n")

# Per ogni rm_id in prediction_mapping, dobbiamo creare le stesse feature
# utilizzate durante il training, basandoci sui dati storici

pred_data = prediction_mapping.copy()

# ========== STATISTICHE STORICHE PER RM_ID ==========
print("📊 Estrazione statistiche storiche per rm_id...")

# Per ogni rm_id, estrai le statistiche dall'ultimo periodo storico
rm_latest_stats = historical_data.groupby('rm_id').tail(30).groupby('rm_id').agg({
    'net_weight': ['mean', 'median', 'std', 'min', 'max'],
    'supplier_id': lambda x: x.mode()[0] if len(x.mode()) > 0 else x.iloc[-1],
    'product_id': lambda x: x.mode()[0] if len(x.mode()) > 0 else x.iloc[-1],
}).reset_index()

# Flatten columns
rm_latest_stats.columns = ['rm_id', 'recent_avg_weight', 'recent_median_weight', 
                            'recent_std_weight', 'recent_min_weight', 'recent_max_weight',
                            'most_common_supplier', 'most_common_product']

# Merge con prediction data
pred_data = pred_data.merge(rm_latest_stats, on='rm_id', how='left')

print(f"✅ Statistiche rm_id aggiunte")
print(f"   Match rate: {(~pred_data['recent_avg_weight'].isna()).sum() / len(pred_data) * 100:.2f}%")

# ========== TEMPORAL FEATURES ==========
print("\n⏰ Creazione temporal features...")

# Usa forecast_end_date come data di riferimento
pred_data['year'] = pred_data['forecast_end_date'].dt.year
pred_data['month'] = pred_data['forecast_end_date'].dt.month
pred_data['day'] = pred_data['forecast_end_date'].dt.day
pred_data['day_of_week'] = pred_data['forecast_end_date'].dt.dayofweek
pred_data['quarter'] = pred_data['forecast_end_date'].dt.quarter
pred_data['week_of_year'] = pred_data['forecast_end_date'].dt.isocalendar().week
pred_data['is_weekend'] = pred_data['day_of_week'].isin([5, 6]).astype(int)
pred_data['is_month_start'] = pred_data['forecast_end_date'].dt.is_month_start.astype(int)
pred_data['is_month_end'] = pred_data['forecast_end_date'].dt.is_month_end.astype(int)
pred_data['is_quarter_start'] = pred_data['forecast_end_date'].dt.is_quarter_start.astype(int)
pred_data['is_quarter_end'] = pred_data['forecast_end_date'].dt.is_quarter_end.astype(int)

# Ciclici
pred_data['month_sin'] = np.sin(2 * np.pi * pred_data['month'] / 12)
pred_data['month_cos'] = np.cos(2 * np.pi * pred_data['month'] / 12)
pred_data['day_sin'] = np.sin(2 * np.pi * pred_data['day_of_week'] / 7)
pred_data['day_cos'] = np.cos(2 * np.pi * pred_data['day_of_week'] / 7)
pred_data['week_sin'] = np.sin(2 * np.pi * pred_data['week_of_year'] / 52)
pred_data['week_cos'] = np.cos(2 * np.pi * pred_data['week_of_year'] / 52)

# Tempo dall'ultimo dato storico
last_historical_date = historical_data['date_arrival'].max()
pred_data['days_since_start'] = (pred_data['forecast_end_date'] - historical_data['date_arrival'].min()).dt.days
pred_data['weeks_since_start'] = pred_data['days_since_start'] / 7

print("✅ Temporal features create")

# ========== SUPPLIER & PRODUCT IDs ==========
print("\n👨‍🏭 Assegnazione supplier_id e product_id...")

pred_data['supplier_id'] = pred_data['most_common_supplier']
pred_data['product_id'] = pred_data['most_common_product']

# ========== STATISTICHE AGGREGATE STORICHE ==========
print("\n📊 Merge con statistiche aggregate storiche...")

# Estrai le statistiche aggregate finali dal dataset storico
# (queste sono già presenti nel dataset processato)
stat_cols = [col for col in historical_data.columns if any(x in col for x in 
             ['supplier_mean', 'supplier_median', 'supplier_std', 'supplier_cv', 'supplier_count',
              'rm_mean', 'rm_std', 'rm_count', 'rm_cv',
              'product_mean', 'product_std', 'product_count', 'product_cv'])]

# Per ogni rm_id, prendi le ultime statistiche disponibili
latest_stats = historical_data.groupby('rm_id').last()[stat_cols].reset_index()
pred_data = pred_data.merge(latest_stats, on='rm_id', how='left')

print("✅ Statistiche aggregate aggiunte")

# ========== LAG FEATURES (dalle ultime ricezioni) ==========
print("\n⏳ Creazione lag features...")

# Per ogni rm_id, estrai le ultime N ricezioni
lag_features = {}
for lag in [1, 2, 3, 5, 7, 14]:
    lag_col = f'weight_lag_{lag}'
    lag_dict = historical_data.groupby('rm_id')['net_weight'].apply(
        lambda x: x.iloc[-lag] if len(x) >= lag else x.mean()
    ).to_dict()
    pred_data[lag_col] = pred_data['rm_id'].map(lag_dict)

# Rolling statistics
for window in [3, 7, 14, 30]:
    rolling_mean_col = f'rolling_mean_{window}'
    rolling_mean_dict = historical_data.groupby('rm_id')['net_weight'].apply(
        lambda x: x.tail(window).mean()
    ).to_dict()
    pred_data[rolling_mean_col] = pred_data['rm_id'].map(rolling_mean_dict)
    
    rolling_std_col = f'rolling_std_{window}'
    rolling_std_dict = historical_data.groupby('rm_id')['net_weight'].apply(
        lambda x: x.tail(window).std()
    ).to_dict()
    pred_data[rolling_std_col] = pred_data['rm_id'].map(rolling_std_dict)
    
    rolling_min_col = f'rolling_min_{window}'
    rolling_min_dict = historical_data.groupby('rm_id')['net_weight'].apply(
        lambda x: x.tail(window).min()
    ).to_dict()
    pred_data[rolling_min_col] = pred_data['rm_id'].map(rolling_min_dict)
    
    rolling_max_col = f'rolling_max_{window}'
    rolling_max_dict = historical_data.groupby('rm_id')['net_weight'].apply(
        lambda x: x.tail(window).max()
    ).to_dict()
    pred_data[rolling_max_col] = pred_data['rm_id'].map(rolling_max_dict)

print("✅ Lag e rolling features create")

# ========== INTERACTION FEATURES ==========
print("\n🔀 Creazione interaction features...")

# Combinazioni
pred_data['supplier_rm_freq'] = 0  # Default
pred_data['supplier_product_freq'] = 0
pred_data['rm_product_freq'] = 0
pred_data['supplier_rm_mean'] = pred_data['recent_avg_weight']
pred_data['supplier_product_mean'] = pred_data['recent_avg_weight']
pred_data['rm_product_mean'] = pred_data['recent_avg_weight']

# Deviazioni e ratio
pred_data['deviation_from_supplier_mean'] = 0
pred_data['deviation_from_rm_mean'] = 0
pred_data['deviation_from_product_mean'] = 0
pred_data['weight_to_supplier_mean_ratio'] = 1.0
pred_data['weight_to_rm_mean_ratio'] = 1.0
pred_data['weight_to_product_mean_ratio'] = 1.0

# Trend
pred_data['trend_3_vs_14'] = pred_data['rolling_mean_3'] / (pred_data['rolling_mean_14'] + 1e-6)
pred_data['trend_7_vs_30'] = pred_data['rolling_mean_7'] / (pred_data['rolling_mean_30'] + 1e-6)
pred_data['momentum_1_3'] = pred_data['weight_lag_1'] - pred_data['weight_lag_3']
pred_data['momentum_3_7'] = pred_data['weight_lag_3'] - pred_data['weight_lag_7']

# Recency
pred_data['days_since_last_receival'] = (pred_data['forecast_end_date'] - last_historical_date).dt.days
pred_data['supplier_avg_days_between'] = 7.0  # Default

print("✅ Interaction features create")

# ========== ALTRE FEATURE MANCANTI ==========
# Aggiungi le feature che potrebbero mancare rispetto al training set
missing_features = [col for col in feature_cols if col not in pred_data.columns]

if missing_features:
    print(f"\n⚠️ Aggiungo {len(missing_features)} feature mancanti con valori di default...")
    for col in missing_features:
        # Valori di default sensati
        if 'ratio' in col or 'trend' in col:
            pred_data[col] = 1.0
        elif 'cv' in col or 'std' in col:
            pred_data[col] = 0.0
        elif 'encoded' in col:
            pred_data[col] = 0
        else:
            pred_data[col] = 0.0

print(f"\n✅ Feature engineering completato!")
print(f"   Shape: {pred_data.shape}")

## 4. Gestione Valori Mancanti e Preparazione Feature Matrix

In [ ]:
print("🔧 Gestione valori mancanti...\n")

# Riempi NaN con valori sensati (come fatto nel training)
lag_cols = [col for col in pred_data.columns if 'lag_' in col or 'rolling_' in col or 'momentum_' in col]
for col in lag_cols:
    if col in pred_data.columns:
        pred_data[col] = pred_data[col].fillna(pred_data[col].median())

ratio_cols = [col for col in pred_data.columns if 'ratio' in col or 'trend_' in col]
for col in ratio_cols:
    if col in pred_data.columns:
        pred_data[col] = pred_data[col].fillna(1.0)

cv_cols = [col for col in pred_data.columns if '_cv' in col]
for col in cv_cols:
    if col in pred_data.columns:
        pred_data[col] = pred_data[col].fillna(0)

# Riempi tutti i rimanenti NaN con 0
pred_data = pred_data.fillna(0)

print(f"✅ Valori mancanti gestiti")
print(f"   NaN rimanenti: {pred_data[feature_cols].isnull().sum().sum()}")

# Prepara feature matrix
X_pred = pred_data[feature_cols].values

print(f"\n📊 Feature matrix per predizione:")
print(f"   Shape: {X_pred.shape}")
print(f"   Samples: {X_pred.shape[0]}")
print(f"   Features: {X_pred.shape[1]}")

# Verifica finale
if np.isnan(X_pred).any():
    print(f"\n⚠️ WARNING: Ci sono ancora NaN nella feature matrix!")
    nan_cols = [feature_cols[i] for i in range(len(feature_cols)) if np.isnan(X_pred[:, i]).any()]
    print(f"   Colonne con NaN: {nan_cols[:10]}")
else:
    print(f"\n✅ Nessun NaN nella feature matrix!")

## 5. Generazione Predizioni con Ensemble

In [ ]:
print("🔮 Generazione predizioni con ensemble...\n")

# ========== LightGBM ==========
print("🌟 Predicting con LightGBM...")
lgb_predictions = lgb_model.predict(X_pred)
print(f"   Min: {lgb_predictions.min():.2f} | Max: {lgb_predictions.max():.2f} | Mean: {lgb_predictions.mean():.2f}")

# ========== CatBoost ==========
print("\n🐈 Predicting con CatBoost...")
catboost_predictions = catboost_model.predict(X_pred)
print(f"   Min: {catboost_predictions.min():.2f} | Max: {catboost_predictions.max():.2f} | Mean: {catboost_predictions.mean():.2f}")

# ========== XGBoost ==========
print("\n🚀 Predicting con XGBoost...")
xgb_predictions = xgb_model.predict(X_pred)
print(f"   Min: {xgb_predictions.min():.2f} | Max: {xgb_predictions.max():.2f} | Mean: {xgb_predictions.mean():.2f}")

# ========== Neural Network ==========
print("\n🧠 Predicting con Neural Network...")
X_pred_scaled = scaler.transform(X_pred)
X_pred_tensor = torch.FloatTensor(X_pred_scaled).to(device)

with torch.no_grad():
    nn_predictions = nn_model(X_pred_tensor).cpu().numpy()

print(f"   Min: {nn_predictions.min():.2f} | Max: {nn_predictions.max():.2f} | Mean: {nn_predictions.mean():.2f}")

# ========== ENSEMBLE ==========
print("\n🎭 Creazione ensemble...")

ensemble_predictions = (
    ensemble_weights['LightGBM'] * lgb_predictions +
    ensemble_weights['CatBoost'] * catboost_predictions +
    ensemble_weights['XGBoost'] * xgb_predictions +
    ensemble_weights['NeuralNet'] * nn_predictions
)

# Clip valori negativi a 0 (non può esistere peso negativo)
negative_count = (ensemble_predictions < 0).sum()
if negative_count > 0:
    print(f"\n⚠️ Clipping {negative_count} predizioni negative a 0")
    ensemble_predictions = np.maximum(ensemble_predictions, 0)

print(f"\n✅ Ensemble predictions:")
print(f"   Min: {ensemble_predictions.min():.2f}")
print(f"   Max: {ensemble_predictions.max():.2f}")
print(f"   Mean: {ensemble_predictions.mean():.2f}")
print(f"   Median: {np.median(ensemble_predictions):.2f}")
print(f"   Std: {ensemble_predictions.std():.2f}")

## 6. Creazione Submission File

In [ ]:
print("📝 Creazione submission file...\n")

# Crea dataframe submission
submission = pd.DataFrame({
    'ID': pred_data['ID'],
    'predicted_weight': ensemble_predictions
})

# Verifica formato
print(f"✅ Submission creata:")
print(f"   Shape: {submission.shape}")
print(f"   Colonne: {submission.columns.tolist()}")
print(f"   IDs unici: {submission['ID'].nunique()}")
print(f"\n📋 Prime righe:")
display(submission.head(10))

# Verifica che tutti gli ID siano presenti
required_ids = set(prediction_mapping['ID'])
submission_ids = set(submission['ID'])

if required_ids == submission_ids:
    print(f"\n✅ Tutti gli ID richiesti sono presenti!")
else:
    missing = required_ids - submission_ids
    extra = submission_ids - required_ids
    if missing:
        print(f"\n⚠️ ID mancanti: {len(missing)}")
    if extra:
        print(f"\n⚠️ ID extra: {len(extra)}")

# Verifica valori
if submission['predicted_weight'].isnull().any():
    print(f"\n❌ WARNING: Ci sono valori NaN nelle predizioni!")
    print(f"   Count: {submission['predicted_weight'].isnull().sum()}")
else:
    print(f"\n✅ Nessun valore NaN nelle predizioni")

if (submission['predicted_weight'] < 0).any():
    print(f"\n❌ WARNING: Ci sono valori negativi!")
    print(f"   Count: {(submission['predicted_weight'] < 0).sum()}")
else:
    print(f"\n✅ Nessun valore negativo")

# Salva submission
output_path = '../data/submission_advanced_ensemble.csv'
submission.to_csv(output_path, index=False)

print(f"\n💾 Submission salvata in: {output_path}")
print(f"\n🎉 Processo completato con successo!")

## 7. Statistiche e Visualizzazioni Finali

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

print("📊 Analisi finale delle predizioni...\n")

# Statistiche dettagliate
print(f"📈 Statistiche predizioni:")
print(submission['predicted_weight'].describe())

# Confronto con dati storici
historical_weights = historical_data['net_weight']
print(f"\n📊 Confronto con dati storici:")
print(f"   Historical mean: {historical_weights.mean():.2f}")
print(f"   Predicted mean: {submission['predicted_weight'].mean():.2f}")
print(f"   Difference: {(submission['predicted_weight'].mean() - historical_weights.mean()):.2f}")

# Visualizzazioni
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. Distribuzione predizioni
axes[0, 0].hist(submission['predicted_weight'], bins=100, edgecolor='black', alpha=0.7, label='Predicted')
axes[0, 0].hist(historical_weights, bins=100, edgecolor='black', alpha=0.5, label='Historical')
axes[0, 0].set_xlabel('Weight (kg)')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].set_title('Distribution: Predicted vs Historical')
axes[0, 0].legend()
axes[0, 0].grid(alpha=0.3)

# 2. Boxplot
data_to_plot = [historical_weights, submission['predicted_weight']]
axes[0, 1].boxplot(data_to_plot, labels=['Historical', 'Predicted'])
axes[0, 1].set_ylabel('Weight (kg)')
axes[0, 1].set_title('Boxplot Comparison')
axes[0, 1].grid(alpha=0.3)

# 3. Predizioni per rm_id (top 10 rm_ids)
top_rm_ids = pred_data['rm_id'].value_counts().head(10).index
for rm_id in top_rm_ids[:5]:  # Solo primi 5 per leggibilità
    rm_preds = submission[pred_data['rm_id'] == rm_id]['predicted_weight']
    axes[1, 0].plot(rm_preds.values[:50], alpha=0.7, label=f'rm_id {rm_id}')

axes[1, 0].set_xlabel('Sample Index')
axes[1, 0].set_ylabel('Predicted Weight (kg)')
axes[1, 0].set_title('Predictions by rm_id (Top 5, first 50 samples)')
axes[1, 0].legend()
axes[1, 0].grid(alpha=0.3)

# 4. Percentili
percentiles = [10, 25, 50, 75, 90, 95, 99]
hist_percentiles = [np.percentile(historical_weights, p) for p in percentiles]
pred_percentiles = [np.percentile(submission['predicted_weight'], p) for p in percentiles]

x = np.arange(len(percentiles))
width = 0.35

axes[1, 1].bar(x - width/2, hist_percentiles, width, label='Historical', alpha=0.8)
axes[1, 1].bar(x + width/2, pred_percentiles, width, label='Predicted', alpha=0.8)
axes[1, 1].set_xlabel('Percentile')
axes[1, 1].set_ylabel('Weight (kg)')
axes[1, 1].set_title('Percentile Comparison')
axes[1, 1].set_xticks(x)
axes[1, 1].set_xticklabels([f'{p}%' for p in percentiles])
axes[1, 1].legend()
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.savefig('../data/submission_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\n💾 Grafici salvati in: ../data/submission_analysis.png")
print(f"\n🏁 Analisi completata!")